# Reset properties in NetCDF mapfile
* Velocities
* Concentrations
* Turbulent energy and dissipation
* Vertical and horizontal eddy viscosity and diffusivity
* Transport layer composition

In [1]:
import xarray as xr
from JulesD3D.processNetCDF import fixMeshGrid, addUnderlayerCoords, makeVelocity
import numpy as np
from IPython.display import Markdown as md

In [3]:
filename = '/Users/julesblom/ThesisResults/trim-36km_200m_W60ChannelRun01_compressed.nc' # Puts all of DP_BEDLYR at 0.5 m?, which is the height of the transport layer
 
trim = xr.open_dataset(filename)

In [4]:
sand = 0 
silt = 1
last_timestep = -1

if silt == sand:
    raise Exception("You stupid")

In [6]:
zeros_interfaces_shape = trim.W.isel(time=last_timestep).shape
zeros_centers_shape = trim.V1.isel(time=last_timestep).shape

In [17]:
all_zeros_centers = np.zeros(zeros_centers_shape)
all_zeros_interfaces = np.zeros(zeros_interfaces_shape)

## Reset velocity

In [16]:
trim['V1'][last_timestep] = all_zeros_centers      
trim['U1'][last_timestep] = all_zeros_centers
trim['WPHY'][last_timestep] = all_zeros_centers
trim['W'][last_timestep] = all_zeros_interfaces

## Reset Eddies

In [18]:
trim['VICWW'][last_timestep] = all_zeros_interfaces # Vertical eddy viscosity-3D
trim['DICWW'][last_timestep] = all_zeros_interfaces # Vertical eddy diffusivity-3D
trim['VICUV'][last_timestep] = all_zeros_centers    # Horizontal eddy viscosity

## Reset Water Level

In [11]:
water_level_shape = trim.S1.isel(time=0).shape
reset_water_level = np.zeros(water_level_shape)

trim['S1'][last_timestep] = reset_water_level

## Reset concentrations

In [19]:
test_conc_shape = trim.R1.isel(time=last_timestep).shape
reset_concentrations = np.zeros(test_conc_shape)

In [20]:
trim['R1'][last_timestep] = reset_concentrations

## Reset density

In [22]:
initial_density = float(trim.RHOCONST.values)
reset_densities = np.ones(zeros_centers_shape) * initial_density

trim['RHO'][last_timestep] = reset_densities

## Reset transport layer

### Volume fraction

In [23]:
vol_frac_sand_transport_layer_end = trim.LYRFRAC.isel(time=last_timestep, LSEDTOT=sand, nlyr=0)
vol_frac_silt_transport_layer_end = trim.LYRFRAC.isel(time=last_timestep, LSEDTOT=silt, nlyr=0)

In [24]:
# Replace transport layer with 50%/50% composition
replaced_vol_frac_sand = vol_frac_sand_transport_layer_end.where(vol_frac_sand_transport_layer_end.values == 0, 0.5)
replaced_vol_frac_silt = vol_frac_silt_transport_layer_end.where(vol_frac_silt_transport_layer_end.values == 0, 0.5)

In [27]:
# Reset volume composition in transport layer at final timestep
trim.LYRFRAC[last_timestep, sand, 0] = replaced_vol_frac_sand.values
trim.LYRFRAC[last_timestep, silt, 0] = replaced_vol_frac_silt.values

### Mass of sediment

In [28]:
# for 50 cm thick transport layer!
initial_sand_mass = 16000 # [kg/m2]
initial_silt_mass = 5000  # [kg/m2]

In [29]:
mass_sand_transport_layer_end = trim.MSED.isel(time=last_timestep, LSEDTOT=sand, nlyr=0)
mass_silt_transport_layer_end = trim.MSED.isel(time=last_timestep, LSEDTOT=silt, nlyr=0)

replaced_mass_sand = mass_sand_transport_layer_end.where(mass_sand_transport_layer_end_copy.values == 0, initial_sand_mass)
replaced_mass_silt = mass_silt_transport_layer_end.where(mass_silt_transport_layer_end_copy.values == 0, initial_silt_mass)

In [ ]:
# Reset mass of sediment in transport layer at final timestep
trim.MSED[last_timestep, sand, 0] = replaced_mass_sand.values
trim.MSED[last_timestep, silt, 0] = replaced_mass_silt.values

## Write to NetCDF

In [ ]:
new_filename = "/Users/julesblom/ThesisResults/MultiTest/trim-36km_200m_W60ChannelRun09_resetunderlayer.nc"
trim.load().to_netcdf(new_filename, mode='w', engine='netcdf4', format='NETCDF3_64BIT') 